# Comparison of IV and CCH method on hippocampal data with optical stimulation of pyramidal neurons

Data kindly provided by Sam McKenzie and Daniel Fine English.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from method import IV
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec
import numpy as np
import quantities as pq
import seaborn as sns
import sys
sys.path.append('../exana/'),
from exana.stimulus import plot_psth
from exana.statistics.plot import plot_xcorr, plot_autocorr
from exana.statistics.tools import ccg_significance, correlogram, ccg

import pandas as pd
import scipy
import neo
import exana

import pdb
from scipy.ndimage.filters import gaussian_filter1d as gaussfilt

import requests
import os

import tools_experimentaldata as tls_exp

from tools_plot import savefig, fix_figure, set_style, despine
import statsmodels.api as sm
from scipy.cluster.hierarchy import dendrogram, linkage  
from scipy import fftpack
from exana.statistics.tools import hollow_kernel as hk
from itertools import chain
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)


ModuleNotFoundError: No module named 'tools_analysis'

In [ ]:
def set_style(style='article', sns_style='white', w=1, h=1):
    sdict = {
        'article': {
            # (11pt font = 360pt, 4.98) (10pt font = 345pt, 4.77)                                                                                                                                                                                                                         
            'figure.figsize' : (4.98 * w, 2 * h),
            'figure.autolayout': False,
            'lines.linewidth': 2,
            'font.size'      : 11,
            'legend.frameon' : False,
            'legend.fontsize': 11,
            'font.family'    : 'serif',
            'text.usetex'    : True
        },
        'notebook': {
            'figure.figsize' : (16, 9),
            'axes.labelsize' : 50,
            'lines.linewidth': 4,
            'lines.markersize': 20,
            'xtick.labelsize': 30,
            'ytick.labelsize': 30,
            'axes.titlesize' : 20,
            'font.size'      : 20,
            'legend.frameon' : False,
            'legend.fontsize': 35,
            'font.family'    : 'serif',
            'text.usetex'    : True
        }
    }
    rc = sdict[style]
    plt.rcParams.update(rc)
    sns.set(rc=rc, style=sns_style,
            color_codes=True)
set_style('article')

In [ ]:
# Load params
from params_experimentaldata import *
# update figure settings



In [ ]:
units_db = tls_exp.load_unitlabels('optoLabels.mat', data_dir)
df_tmp = units_db.drop_duplicates(['animal', 'date'])
relevant_data = df_tmp.groupby('animal')['date'].apply(list).to_dict()

for animal_i in relevant_data.keys():
    for date_i in relevant_data[animal_i]:
        for entry in blk_blacklist:
            if animal_i == entry['animal'] and date_i == entry['date']:
                relevant_data[animal_i].remove(entry['date'])
            

In [ ]:
relevant_data

In [ ]:
load_orig_files = True

if load_orig_files:
    tls_exp.download_files_by_dict(relevant_data,
                                   data_dir,
                                   n_shanks,
                                   files_ext_general,
                                   files_ext_by_shank,
                                   link_db)

    blks = tls_exp.create_neo_structure(relevant_data,
                                        data_dir,
                                        n_shanks,
                                        sampling_rate,
                                        unit_spiketime)

    tls_exp.add_stimulation_data_to_blocks(blks)
    tls_exp.annotate_units_from_db(units_db, blks)
    for blk in blks:
        animal = blk.annotations['animal']
        date = blk.annotations['date']
        nio = neo.io.PickleIO(data_dir + 'neo_files/' + animal + '_' + date + '.pckl')
        nio.write_block(blk)

else:
    blks = []
    for animal in relevant_data.keys():
        for date in relevant_data[animal]:
            nio = neo.io.PickleIO(data_dir + 'neo_files/' + animal + '_' + date + '.pckl')
            blk = nio.read_block()
            blks.append(blk)

In [ ]:
blks = tls_exp.select_blocks_upon_stimtype(blks,
    stimtype='pulse', min_intens=1)

## Determine which stimulations intensities have a significant effect  on units
We a) group very similar stimulations intensities and b) test whether intensity group has a significant effect on increasing spiking probability of any of the given units by convolving stimulation onsets with spike train.
We use the activity before a stimulation as baseline.


In [ ]:
blks = tls_exp.group_stimulations(blks,
                                  sep_bins,
                                  sep_kernel_width,
                                  sep_threshold)

In [ ]:
df_stim = tls_exp.find_significant_stimulations(blks,
   stimccg_binsize,
   stimccg_limit,
   stimccg_pthres,
   condition_annot_unit={'tagged': True})

### Example plot of  stimulation response

In [ ]:
fig, ax = plt.subplots(1)
row_sel = 2
ln0 = ax.plot(df_stim.loc[row_sel]['bins'],
        df_stim.loc[row_sel]['cch'],
       label='count')
ln1 = ax.axhline(df_stim.loc[row_sel]['rate_baseline'],
           label='baseline',
          linestyle='--')
ax.set_xlabel(r'$\Delta t$')
ax.set_ylabel(r'count')
ax2 = ax.twinx()
ln2 = ax2.plot(df_stim.loc[row_sel]['bins'],
         df_stim.loc[row_sel]['pfast'], c='r',
        label='prob')
ln3 = ax2.axhline(stimccg_pthres, c='r', linestyle='--',alpha=0.5, label='sign. level')
ax2.set_ylabel(r'prob')

ax2.set_ylim(0, 0.1)
ax.set_xlim(-50., 50.)
ax2.set_xlim(-50., 50.)

ln = ln0+[ln1]+ln2+[ln3]
labs = [l.get_label() for l in ln]
ax.legend(ln, labs, loc=0)
ax.set_title('Example CCH between stimulus onset and spikes')
plt.show()

## Overview of stimulation intensities
For each unit, we show the strongest available stimulation.

In [ ]:

group_obj = df_stim.groupby(['animal', 'date', 'shank_unit', 'cluster', 'shank_stim'])

idxmax = group_obj['intens_mean'].idxmax()
idxmin = group_obj['intens_mean'].idxmin()


In [ ]:
group_obj['intens_mean'].max()

## Response time by stimulation intensity
We visualize the time it takes for units to show a significant increase in spiking probability for maximal stimulation intensities on same shank vs other shanks.
The last bin, $\Delta t = 25$ ms, includes also larger times

In [ ]:
t_max_same = []
t_max_diff = []
intens_max_same =[]
intens_max_diff =[]
for i in idxmax:
    if df_stim.loc[i]['shank_stim'] == df_stim.loc[i]['shank_unit']:
        t_max_same.append(df_stim.loc[i]['first_bin_sig'])
        intens_max_same.append(df_stim.loc[i]['intens_mean'])
    if df_stim.loc[i]['shank_stim'] != df_stim.loc[i]['shank_unit']:
        t_max_diff.append(df_stim.loc[i]['first_bin_sig'])
        intens_max_diff.append(df_stim.loc[i]['intens_mean'])
t_max_same = np.array(t_max_same)
t_max_diff = np.array(t_max_diff)

In [ ]:
t_min_same = []
t_min_diff = []
intens_min_same =[]
intens_min_diff =[]
for i in idxmin:
    if df_stim.loc[i]['shank_stim'] == df_stim.loc[i]['shank_unit']:
        t_min_same.append(df_stim.loc[i]['first_bin_sig'])
        intens_min_same.append(df_stim.loc[i]['intens_mean'])
    if df_stim.loc[i]['shank_stim'] != df_stim.loc[i]['shank_unit']:
        t_min_diff.append(df_stim.loc[i]['first_bin_sig'])
        intens_min_diff.append(df_stim.loc[i]['intens_mean'])
t_min_same = np.array(t_min_same)
t_min_diff = np.array(t_min_diff)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,3))
bins = np.arange(1.5, 31.5, 3)
hist, bins = np.histogram(t_max_same, bins)
hist = hist / len(t_max_same)
ax.bar(bins[:-1], hist)
despine(ax)
ax.set_title('Time to significant response upon stimulation with maximal intensity on same shank')
ax.set_xlabel(r'$\Delta t$ stimulation onset [ms]')
ax.set_ylabel('Fraction of units')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,3))
bins = np.arange(1.5, 31.5, 3)
hist, bins = np.histogram(t_min_diff, bins)
hist = hist / len(t_min_diff)
ax.bar(bins[:-1], hist)
despine(ax)
ax.set_title('Time to significant response upon stimulation with minimal intensity on different shank')
ax.set_xlabel(r'$\Delta t$ stimulation onset [ms]')
ax.set_ylabel('Fraction of units')
plt.show()

### Observation
Stimulation on a different shank leads only in very few cases to a significant reaction in a reasonable time frame.

## Calculate IV
We calculate the wald estimate ofor those stimulations that show a significant response within the IV window.

In [ ]:
windw = iv_window.rescale(df_stim['first_bin_sig'].values[0].units).magnitude[np.newaxis][0]
df_sigstim = df_stim[df_stim['first_bin_sig'] <= windw]


In [ ]:
df_sigstim['intens_mean'] = df_sigstim['intens_mean'].astype(int)
group_sigstim = df_sigstim.groupby(['animal', 'date', 'shank_unit', 'cluster', 'shank_stim'])


In [ ]:
df_iv = tls_exp.calculate_iv_sigstim(
    blks,
    df_sigstim,
    iv_min_n_stim,
    iv_window,
    iv_ltnc,
    condition_annot_pre={'tagged': True},
    condition_annot_post={'tagged': False})

### Calculate CCH estimate
We calculate the so called transmission probability, the CCH estimate of synaptic coupling.

In [ ]:
df_cch_all = tls_exp.calculate_transmission_prob(
    blks,
    ccg_time_limit,
    ccg_binsize,
    ccg_hollow_fraction,
    ccg_width,
    ccg_sig_level_causal,
    ccg_sig_level_fast,
    ccg_peak_wndw,
    condition_annot_pre={'tagged': True},
    condition_annot_post={'tagged': False})
df_cch_all.rename(columns={'transprob': 'transproball',
                            'bool_cnnctd': 'boolcnnctdall'}, inplace=True)

For curiosity we do it also for only spikes that arised spontaneously and the first evoked spikes

In [ ]:
blks_nostim = tls_exp.keep_spikes_by_stim(blks, keep='nostim')

In [ ]:
df_cch_nostim = tls_exp.calculate_transmission_prob(
    blks_nostim,
    ccg_time_limit,
    ccg_binsize,
    ccg_hollow_fraction,
    ccg_width,
    ccg_sig_level_causal,
    ccg_sig_level_fast,
    ccg_peak_wndw, 
    condition_annot_pre={'tagged': True},
    condition_annot_post={'tagged': False})
df_cch_nostim.rename(
    columns={'transprob': 'transprobspont',
             'bool_cnnctd': 'boolcnnctdspont'},
    inplace=True)

In [ ]:
    blks_stim = tls_exp.select_only_first_spike(
        blks,
        condition_annot_unit={'tagged': True})

In [ ]:
df_cch_stim = tls_exp.calculate_transmission_prob(
    blks_stim,
    ccg_time_limit,
    ccg_binsize,
    ccg_hollow_fraction,
    ccg_width,
    ccg_sig_level_causal,
    ccg_sig_level_fast,
    ccg_peak_wndw,
    condition_annot_pre={'tagged': True},
    condition_annot_post={'tagged': False})
df_cch_stim.rename(columns={'transprob': 'transprobevoked',
                            'bool_cnnctd': 'boolcnnctdevoked'}, inplace=True)

In [ ]:
df_cch = pd.merge(df_iv, df_cch_nostim,
    on=['animal','date',
        'shank_pre', 'cluster_pre',
       'shank_post','cluster_post'])
df_cch = pd.merge(df_cch, df_cch_stim,
    on=['animal','date',
        'shank_pre', 'cluster_pre',
        'shank_post','cluster_post'])
#df_cch = pd.merge(df_iv, df_cch_stim,
#    on=['animal','date',
#        'shank_pre', 'cluster_pre',
#       'shank_post','cluster_post'])
df_cch = pd.merge(df_cch, df_cch_all,
    on=['animal','date',
        'shank_pre', 'cluster_pre',
        'shank_post','cluster_post'])

In [ ]:
fig, ax = plt.subplots(1, figsize=(5,5))

#ax.set(xscale="log", yscale="log")

ax.scatter(df_cch.loc[df_cch['boolcnnctdall']==True]['transproball'],
           df_cch.loc[df_cch['boolcnnctdall']==True]['ivwald'],
          c='b',
          alpha=0.4)
#ax.scatter(df_cch['transproball'],
#           df_cch['ivwald'],
#          c='r',
#          alpha=0.1)
ax.set_xlabel(r'CCH')
ax.set_ylabel(r'IV')
ax.set_xlim([-0.0001, 0.011])
#ax.set_ylim([-0.03, 0.125])
despine(ax)
ax.set_title('Comparison IV and CCH on all spikes')
plt.show()
fig.tight_layout(rect=[0, 0.00, 1, 1])
#fig.savefig('manuscript/Optodata_comparisonIV_CCH.svg')


In [ ]:
fig, ax = plt.subplots(1, figsize=(5,5))

#ax.set(xscale="log", yscale="log")

ax.scatter(df_cch.loc[df_cch['boolcnnctdspont']==True]['transprobspont'],
           df_cch.loc[df_cch['boolcnnctdspont']==True]['ivwald'],
          c='b',
          alpha=0.4)
ax.set_xlim([-0.0001, 0.011])
ax.set_ylim([-0.03, 0.125])
ax.set_xlabel(r'CCH spontaneous')
ax.set_ylabel(r'iv wald')
ax.set_title('Comparison IV and CCH on spontaneous spikes')
despine(ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(5,5))

#ax.set(xscale="log", yscale="log")

ax.scatter(df_cch.loc[df_cch['boolcnnctdevoked']==True]['transprobevoked'],
           df_cch.loc[df_cch['boolcnnctdevoked']==True]['ivwald'],
          c='b',
          alpha=0.4)
ax.set_xlim([-0.0001, 0.011])
ax.set_ylim([-0.03, 0.125])
ax.set_xlabel(r'CCH spontaneous')
ax.set_ylabel(r'iv wald')
ax.set_title('Comparison IV and CCH on first evoked spike only')
despine(ax)
plt.show()

In [ ]:
group_df_pre = df_cch[
    ['animal', 'date', 'shank_pre', 'cluster_pre']
].groupby(['animal', 'date', 'shank_pre']).nunique()
group_df_pre

In [ ]:
group_df_post = df_cch[
    ['animal', 'date', 'shank_post', 'cluster_post']
].groupby(
    ['animal', 'date', 'shank_post'])
group_df_post.nunique()

In [ ]:
#with open('session_expdata.tex','w') as tf:
#    tf.write(group_sigstim['intens_mean'].unique().to_latex())
group_sigstim['intens_mean'].unique()

## Correlation of IV and CCH estimate

In [ ]:
fig, ax = plt.subplots(1,1)
res =tls_exp.regplot('transproball', 'ivwald',
                df_cch.loc[df_cch['boolcnnctdall']==True],
                sm.OLS, colorbar=False, xlabel=r'CCH',
                ylabel=r'IV', ax=ax)
print('pValue: ' + str(res.f_pvalue))


In [ ]:
df_cch.corr()

### Observation
We find that there is a weak correlation between iv and the cch estimate of $0.24$.

### Autocorrelation of presynaptic units
Because of slow stimulation onset we used a relatively long iv window.
We want to see how this compares to the refractory period of pyramidal cells.

In [ ]:
autocorr_dict = {}
units_pre = df_cch.loc[df_cch['boolcnnctdall']==True].groupby(
    ['animal', 'date', 'shank_pre', 'cluster_pre']
).apply(list).to_dict()
for animal, date, shank_pre, cluster_pre in units_pre.keys():
    for blk in blks:
        units = blk.channel_indexes[0].children
        if blk.annotations['date'] == date and blk.annotations['animal'] == animal:
            unit_i = [unit for unit in units if
                      unit.annotations['shank'] == shank_pre and
                      unit.annotations['cluster'] == cluster_pre][0]
            spktr=unit_i.spiketrains[0]
            cnt, bins_autocorr = correlogram(
                spktr, auto=True,
                limit=autocorr_limit,
                binsize=autocorr_binsize,
                density=True)
            autocorr_dict[animal + '_' + date + '_' 
                          +str(shank_pre) +'_'+ str(cluster_pre)] = cnt

In [ ]:
autocorr = np.array(list(chain(autocorr_dict.values())))

In [ ]:
fig, ax = plt.subplots(1)
autocorr_mean = np.mean(autocorr, axis=0)
autocorr_std = np.std(autocorr, axis=0)

ax.plot(bins_autocorr, autocorr_mean)
ax.plot(bins_autocorr, autocorr_mean+autocorr_std, alpha=0.1)
ax.plot(bins_autocorr, autocorr_mean-autocorr_std, alpha=0.1)

ax.set_ylabel(r'Mean of normalized counts')
ax.set_xlabel(r'Offset')
despine(ax)
savefig(fig, fname='Optodata_autocorrelation.svg')

### Observation
The average of the scaled autocorrelogram indicates a refractory period of around $4 ms$

## Bootstrapping
To test the uncertainty of each IV and CCH estimate, we perform bootstrapping with a sample size of $1000$

#### CCH
We use the additive property of the cross correlation function.
We calculate the CCH on chunks of spike trains.
We then randomly pick chunks with replacement, add them up and perform CCH estimation.

#### IV
We first find all onsets and classify whether they contain a spike or not.
Then we randomly pick onsets with replacement.

In [ ]:
# Consider only significant connections
# for bootstrapping
conns_sel = list(
    df_cch.loc[df_cch['boolcnnctdall']==True].groupby(
    ['animal', 'date', 'shank_pre', 'cluster_pre', 'shank_post', 'cluster_post']
    ).apply(list).to_dict())

In [ ]:
calculate_df_btstrp_cch = True

if calculate_df_btstrp_cch:
    df_btstrp_cch = tls_exp.bootstrap_cch(
            blks, btstrp_n,
            btstrp_binsize,                                                                                                                                                                                            
            ccg_binsize,                                                                                                                                                                                               
            ccg_hollow_fraction,                                                                                                                                                                                       
            ccg_peak_wndw,                                                                                                                                                                                             
            ccg_time_limit,
            ccg_width,
            conns_sel=conns_sel)
    df_btstrp_cch.to_pickle('df_btstrp_cch')
else:
    df_btstrp_cch = pd.read_pickle('df_btstrp_cch')

In [ ]:
calculate_df_btstrp_iv = True

if calculate_df_btstrp_iv:
    df_btstrp_iv = tls_exp.bootstrap_iv(
            blks, btstrp_n,                                                                                                                                                                                                   
            df_sigstim,                                                                                                                                                                                                
            iv_min_n_stim,                                                                                                                                                                                             
            iv_window,                                                                                                                                                                                                 
            iv_ltnc,                                                                                                                                                                                                   
            conns_sel=conns_sel)
    df_btstrp_iv.to_pickle('df_btstrp_iv')
else:
    df_btstrp_iv = pd.read_pickle('df_btstrp_iv')

In [ ]:
CI = 66.6
perc_low = (100-CI)/2
perc_high = 100 - perc_low

cch_err_low = []
cch_err_high = []

iv_err_low = []
iv_err_high = []

iv_med = []
cch_med = []

for conn in conns_sel:
    (animal,
     date,
     shank_pre,
     cluster_pre,
     shank_post,
     cluster_post) = conn
    df_conn_iv = df_btstrp_iv.loc[
        (df_btstrp_iv['animal'] == animal) &
        (df_btstrp_iv['date'] == date) &
        (df_btstrp_iv['shank_pre'] == shank_pre) &
        (df_btstrp_iv['cluster_pre'] == cluster_pre) &
        (df_btstrp_iv['shank_post'] == shank_post) &
        (df_btstrp_iv['cluster_post'] == cluster_post)]
    med = np.nanmedian(df_conn_iv['ivwald'])
    err_low = med-np.nanpercentile(df_conn_iv['ivwald'], perc_low)
    err_high = np.nanpercentile(df_conn_iv['ivwald'], perc_high)-med
    iv_med.append(med)
    iv_err_low.append(err_low)
    iv_err_high.append(err_high)


    df_conn_cch = df_btstrp_cch.loc[
        (df_btstrp_cch['animal'] == animal) &
        (df_btstrp_cch['date'] == date) &
        (df_btstrp_cch['shank_pre'] == shank_pre) &
        (df_btstrp_cch['cluster_pre'] == cluster_pre) &
        (df_btstrp_cch['shank_post'] == shank_post) &
        (df_btstrp_cch['cluster_post'] == cluster_post)]
    med = np.nanmedian(df_conn_cch['trans_prob'])
    err_low = med-np.nanpercentile(df_conn_cch['trans_prob'], perc_low)
    err_high = np.nanpercentile(df_conn_cch['trans_prob'], perc_high)-med
    
    cch_err_low.append(err_low)
    cch_err_high.append(err_high)    
    cch_med.append(med)
    
iv_err = np.vstack([iv_err_low, iv_err_high])
iv_med = np.array(iv_med)
cch_err = np.vstack([cch_err_low, cch_err_high])
cch_mean = np.array(cch_mean)

In [ ]:
fig, ax = plt.subplots(1, figsize=(4,4))

#ax.set(xscale="log", yscale="log")
x = cch_mean
y = iv_mean
ax.scatter(x,
           y*0.1,
           c='b',
           alpha=0.6)
ax.errorbar(x,
            y*0.1,
            xerr=cch_err,
            yerr=iv_err*0.1,
            fmt='None',
            ecolor='b',
            alpha=0.2)
#ax.scatter(df_cch['transproball'],
#           df_cch['ivwald'],
#          c='r',
#          alpha=0.1)
ax.set_xlabel(r'CCH')
ax.set_ylabel(r'IV $[10^{-1}]$')
ax.set_xlim([-0.0001, 0.006])
#ax.set_xlim([-0.0001, 0.01])
ax.set_ylim([-0.0065, 0.025])
ax.plot([0,1], [0,1], 'k--', alpha=0.3)
despine(ax)
plt.show()
fig.tight_layout(rect=[0, 0.00, 1, 1])
fig.savefig('manuscript/Optodata_comparisonIV_CCH.svg')


In [ ]:
id_max = np.argmax(x)
print(x[id_max])
print(y[id_max])

In [ ]:
a=np.vstack([iv_err_low, iv_err_high])*0.1
a[0,0:]

In [ ]:
np.mean(df_conn_cch['trans_prob'])

In [ ]:
np.mean(df_conn_iv['ivwald'])

In [ ]:
np.percentile(df_conn_iv['ivwald'], 2.5)

In [ ]:
np.percentile(df_conn_iv['ivwald'], 97.5)

### Uncertainty and iv window length
We observed that having a small window size, results in many units having very few spikes after stimulation.
Therefore we expect that small window size results in higher errorbars

In [ ]:
# test different iv windows for their error
lst_iv_windows = np.array([1.5, 3., 4.5, 6., 7.5, 9.])*pq.s
lst_df_diffwindw = []
for windw in lst_iv_windows:
    df_i = tls_exp.bootstrap_iv(
            blks, btstrp_n,                                                                                                                                                                                                
            df_sigstim,                                                                                                                                                                                                
            iv_min_n_stim,                                                                                                                                                                                             
            windw,                                                                                                                                                                                                 
            iv_ltnc,
            conns_sel=conns_sel)
    lst_df_diffwindw.append(df_i)


In [ ]:
mean_err = []
for i in range(len(lst_iv_windows)):
    df_i = lst_df_diffwindw[i]
    iv_err = []
    for conn in conns_sel:
        (animal,
         date,
         shank_pre,
         cluster_pre,
         shank_post,
         cluster_post) = conn
        df_conn_iv = df_i.loc[
            (df_i['animal'] == animal) &
            (df_i['date'] == date) &
            (df_i['shank_pre'] == shank_pre) &
            (df_i['cluster_pre'] == cluster_pre) &
            (df_i['shank_post'] == shank_post) &
            (df_i['cluster_post'] == cluster_post)]
        iv_err.append(np.nanstd(df_conn_iv['ivwald'])/np.sqrt(len(df_conn_iv['ivwald'])))
    iv_err = np.array(iv_err)
    mean_err.append(np.nanmean(iv_err))

In [ ]:
fig, ax = plt.subplots(1)
ax.plot(lst_iv_windows, mean_err)
ax.set_title('Certainty of IV estimates against IV window length')
ax.set_xlabel(r'IV time window [ms]')
#ax.set_ylabel(r'Mean standard deviation of error')
despine(ax)
fig.savefig('manuscript/Optodata_error_by_IV_window.svg')
